# Epidemics on networks

In [1]:
import numpy
import networkx
import epyc
import epydemic

import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import seaborn

from ipywidgets import interact, fixed
import ipywidgets as widgets

In [2]:
def make_sir(beta, alpha):
    # turn the equations into update functions
    def dS(S, I, R):
        return -beta * S * I
    def dI(S, I, R):
        return beta * S * I - alpha * I
    def dR(S, I, R):
        return alpha * I
    
    # return the three functions
    return (dS, dI, dR)

In [12]:
def epidemic_sir(T, N, pInfected, pInfect, pRemove):
    # create the equations for these parameters
    (dS, dI, dR) = make_sir(pInfect, pRemove)
    
    # initial conditions
    sss = [ N * (1.0 - pInfected) ]   # everyone is initially susceptible...
    iss = [ N * pInfected ]           # ...except for this fraction...
    rss = [ 0 ]                       # ...and no-one starts off removed

    # push the initial conditions through the differential equations
    for t in range(1, T):
        ds = dS(sss[-1], iss[-1], rss[-1])
        di = dI(sss[-1], iss[-1], rss[-1])
        dr = dR(sss[-1], iss[-1], rss[-1])
        sss.append(sss[-1] + ds)
        iss.append(iss[-1] + di)
        rss.append(rss[-1] + dr)
        
    # return the time series
    return (list(range(0, T)), sss, iss, rss)

In [4]:
N = 1000
T = 5000
pInfected = 0.01

In [5]:
class MonitoredSIR(epydemic.SIR):
    '''A monitored epidemic.'''''
    
    # Experimental parameters
    DELTA = "time_delta"                #: Parameter for the time interval for observations.
    
    # Results
    OBSERVATIONS = 'loci_observations'  #: Result holding the times of the observations.
    TIMESERIES = "loci_timeseries"      #: Result holding a dict mapping locus names to a dict of sample time and size.
 
    def __init__(self):
        super(MonitoredSIR, self).__init__()
        
    def reset(self):
        '''Reset the process.'''
        super(MonitoredSIR, self).reset()
        self._timeSeries = None
        
    def build(self, params):
        '''Build the observation process.
        
        :param params: the experimental parameters'''
        super(MonitoredSIR, self).build(params)

        # also monitor other compartments
        self.trackNodesInCompartment(epydemic.SIR.SUSCEPTIBLE)
        self.trackNodesInCompartment(epydemic.SIR.REMOVED)
        
        # post a repeating event to observe the process
        delta = params[self.DELTA]
        self.postRepeatingEvent(delta, delta, None, self.observe)
        
    def observe(self, t, e):
        '''Observe the network.
        
        :param t: the current simulation time
        :param e: the element (ignored)'''

        # if this is the first run, build the initial dicts per locus
        if self._timeSeries is None:
            self._timeSeries = dict()
            self._timeSeries[self.OBSERVATIONS] = []
            for l in self:
                self._timeSeries[l] = []

        # make the observation
        self._timeSeries[self.OBSERVATIONS].append(t)
        for l in self:
            self._timeSeries[l].append(len(self[l]))
        
    def results(self):
        '''Return the time series.
        
        :returns: the results'''
        rc = super(MonitoredSIR, self).results()
        
        # store the series
        rc[self.TIMESERIES] = self._timeSeries
        
        return rc

In [6]:
def network_sir(T, g, pInfected, pInfect, pRemove):
    # create the simulator
    m = MonitoredSIR()
    m.setMaximumTime(T)
    e = epydemic.StochasticDynamics(m, g)

    # set the simulation parameters
    param = dict()
    param[epydemic.SIR.P_INFECTED] = pInfected
    param[epydemic.SIR.P_INFECT] = pInfect
    param[epydemic.SIR.P_REMOVE] = pRemove
    param[MonitoredSIR.DELTA] = T / 100          # take 100 samples
    
    # run the simulation
    rc = e.set(param).run()
    
    # extract the time series, converting them to fractions of the network
    results = e.experimentalResults()[MonitoredSIR.TIMESERIES]
    ts = results[MonitoredSIR.OBSERVATIONS]
    sss = results[epydemic.SIR.SUSCEPTIBLE]
    iss = results[epydemic.SIR.INFECTED]
    rss = results[epydemic.SIR.REMOVED]

    # return the time series
    return(ts, sss, iss, rss)

In [7]:
g = networkx.complete_graph(N)

In [13]:
@interact(pInfected=fixed(pInfected), \
          pInfect=widgets.FloatSlider(min=0.0, max=0.01, step=(0.01 / 20), value=0.004, \
                                      description='$\\beta \\times 1000$', continuous_update=False, readout_format='.4f'), \
          pRemove=widgets.FloatSlider(min=0.0, max=0.004, step=(0.004 / 20), value=0.001, \
                                      description='$\\alpha$', continuous_update=False, readout_format='.4f'))
def interact_networked_sir(pInfected, pInfect, pRemove):
    # run the epidemic equations
    (ts, sss, iss, rss) = epidemic_sir(T, N, pInfected, pInfect / 1000, pRemove)
    
    # run the corresponding simulation
    (sim_ts, sim_sss, sim_iss, sim_rss) = network_sir(T, g, pInfected, pInfect / 1000, pRemove)
    
    # draw the graph
    fig = plt.figure(figsize=(8, 6))
    plt.plot(ts, sss, 'r-', label='suceptible')   # the theoretical results
    plt.plot(ts, iss, 'g-', label='infected')
    plt.plot(sim_ts, sim_sss, 'rx')               # the simulated results
    plt.plot(sim_ts, sim_iss, 'gx')
    plt.title('Progress of epidemic ($\\beta = {b}, \\alpha = {a}$)'.format(b=pInfect, a=pRemove))
    plt.xlabel('$t$')
    plt.xlim([0, T])
    plt.ylabel('population that is...')
    plt.ylim([0.0, N])
    plt.legend(loc='right')
    plt.show()

interactive(children=(FloatSlider(value=0.004, continuous_update=False, description='$\\beta \\times 1000$', m…